In [ ]:
from keras.models import Model
from keras.layers import Input,Conv2D, Conv3D, MaxPooling3D,UpSampling2D, UpSampling3D,MaxPooling2D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate,Add
import keras as Ks
from keras.layers import Input
from glob import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2
from skimage import io
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=1ded895fa7abd909395c59cfcf983d8baa9918092e255b1088516a29fb06d1ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-njbvebop/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ
To: /content/CT_Data.zip
100% 624M/624M [00:04<00:00, 127MB/s]


In [ ]:
from zipfile import ZipFile

file_name="/content/CT_Data.zip"

with ZipFile(file_name, 'r') as ziip:
  ziip.extractall()
  print('woow')

woow


In [ ]:
mask_files = glob('/content/content/drive/MyDrive/CT_Data/Masks/*')
len(mask_files)

7367

In [ ]:
CT = glob('/content/content/drive/MyDrive/CT_Data/CT_images/*')
len(CT)

7367

In [ ]:
df = pd.DataFrame({"image_path": CT, "mask_path":mask_files})


In [ ]:
Xtrain, Xval = train_test_split(df, test_size=0.15,random_state=101)
Xtest, Xval = train_test_split(Xval, test_size=0.5)
print("Train size is {}, valid size is {} & test size is {}".format(len(Xtrain), len(Xval), len(Xtest)))

Train size is 6261, valid size is 553 & test size is 553


In [ ]:
train_ids = list(Xtrain.image_path)
train_mask = list(Xtrain.mask_path)

val_ids = list(Xval.image_path)
val_mask= list(Xval.mask_path)

In [ ]:
def resblock(X, f):
 
    Xcopy = X  
    
    # main path
    X = Conv2D(f, kernel_size=(1,1), kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(f, kernel_size=(3,3), padding='same', kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    
    # shortcut path
    Xcopy = Conv2D(f, kernel_size=(1,1), kernel_initializer='he_normal')(Xcopy)
    Xcopy = BatchNormalization()(Xcopy)
    
    # Adding the output from main path and short path together
    X = Add()([X, Xcopy])
    X = Activation('relu')(X)
    
    return X

def upsample_concat(x, skip):
    '''
    funtion for upsampling image
    '''
    X = UpSampling2D((2,2))(x)
    merge = Concatenate()([X, skip])
    
    return merge

In [ ]:
def tversky(ytrue, ypred):
    ypredpos = K.flatten(ypred)
    ytruepos = K.flatten(ytrue)
    truepos = K.sum(ytruepos * ypredpos)
    falseneg = K.sum(ytruepos * (1-ypredpos))
    falsepos = K.sum((1-ytruepos)*ypredpos)
    alpha = 0.7
    smooth=100
    return (truepos + smooth)/(truepos + alpha*falseneg + (1-alpha)*falsepos + smooth)

def focaltversky(ytrue,ypred):
    ypred = tf.cast(ypred, tf.float32)
    ytrue = tf.cast(ytrue, tf.float32)
    
    pt_1 = tversky(ytrue, ypred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

def tversky_loss(ytrue, ypred):
    return 1 - tversky(ytrue,ypred)

In [ ]:
def Res_Unet_model(inputshape=(256,256,3),filter_num=16,pool_size=(2, 2),n_classes=3):
  Xinput = Input(inputshape)

  conv_1 = Conv2D(2*filter_num, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(Xinput)
  conv_1 = BatchNormalization()(conv_1)
  conv_1 = Conv2D(2*filter_num, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv_1)
  conv_1 = BatchNormalization()(conv_1)
  pool_1 = MaxPool2D(pool_size)(conv_1)


  conv_2 = resblock(pool_1, 4*filter_num)
  pool_2 = MaxPooling2D(pool_size)(conv_2)


  conv_3 = resblock(pool_2, 8*filter_num)
  pool_3 = MaxPooling2D(pool_size)(conv_3)


  conv_4 = resblock(pool_3, 16*filter_num)
  pool_4 = MaxPooling2D(pool_size)(conv_4)


  conv_5 = resblock(pool_4, 32*filter_num)


  up_1 = upsample_concat(conv_5, conv_4)
  up_1 = resblock(up_1, 16*filter_num)


  up_2 = upsample_concat(up_1, conv_3)
  up_2 = resblock(up_2, 8*filter_num)


  up_3 = upsample_concat(up_2, conv_2)
  up_3 = resblock(up_3, 4*filter_num)


  up_4 = upsample_concat(up_3, conv_1)
  up_4 = resblock(up_4, 2*filter_num)


  out = Conv2D(n_classes, (1,1), kernel_initializer='he_normal', padding='same', activation='sigmoid')(up_4)
  Res_Unet_with_att_model=Model(Xinput, out)
  return Res_Unet_with_att_model

In [ ]:
inputshape = (256,256,3)
Res_Unet_model=Res_Unet_model(inputshape)

In [ ]:
Res_Unet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, ids , mask, image_dir = './', batch_size = 16, img_h = 256, img_w = 256, shuffle = True):
    self.ids = ids
    self.mask = mask
    self.image_dir = image_dir
    self.batch_size = batch_size
    self.img_h = img_h
    self.img_w = img_w
    self.shuffle = shuffle
    self.on_epoch_end()
  def __len__(self):
    return int(np.floor(len(self.ids)) / self.batch_size)

  def __getitem__(self, index):
    indexes = self.indexes[index* self.batch_size : (index+1) * self.batch_size]
    list_ids = [self.ids[i] for i in indexes]
    list_mask = [self.mask[i] for i in indexes]
    X, y = self.__data_generation(list_ids, list_mask)
    return X, y

  def on_epoch_end(self):    
    self.indexes = np.arange(len(self.ids))

    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_ids, list_mask):

    X = np.empty((self.batch_size, self.img_h, self.img_w, 3))
    y = np.empty((self.batch_size, self.img_h, self.img_w, 3))

    for i in range(len(list_ids)):
      img_path = str(list_ids[i])
      mask_path = str(list_mask[i])
      img = io.imread(img_path)
      mask = io.imread(mask_path)
      img = cv2.resize(img,(self.img_h,self.img_w))
      img = np.array(img, dtype = np.float64)
      mask = cv2.resize(mask,(self.img_h,self.img_w))
      mask = np.array(mask, dtype = np.float64)
      img -= img.mean()
      img /= img.std()
      mask -= mask.mean()
      mask /= mask.std()
      X[i,] = img
      y[i,] = mask
    y = (y > 0).astype(int)
    return X, y

train_data = DataGenerator(train_ids, train_mask)
val_data = DataGenerator(val_ids, val_mask)

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=20
                             )
checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/CT_res_unet_32/CT_res_unet_32.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.0001,
                              factor=0.2
                             )
filename='/content/drive/MyDrive/CT_res_unet_32/history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

In [ ]:
adam = tf.keras.optimizers.Adam(lr = 0.01, epsilon = 0.1)
Res_Unet_model.compile(optimizer = adam, 
                  loss = focaltversky, 
                  metrics = [tversky,Recall(), Precision(), MeanIoU(num_classes=3)]
                 )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
Res_Unet_model.load_weights("/content/drive/MyDrive/CT_res_unet_32/CT_res_unet_32.h5")

In [ ]:
def prediction(test, model_seg):
    
    mask, image_id, mask_path =  [], [],[]
    
    for i in test.image_path:
        img = io.imread(i)        
        img = img *1./255.
        img = cv2.resize(img, (256,256))
        img = np.array(img, dtype=np.float64)
        img = np.reshape(img, (1,256,256,3))

        X = np.empty((1,256,256,3))
        
        img = io.imread(i)
        img = cv2.resize(img, (256,256))
        img = np.array(img, dtype=np.float64)
        
        img -= img.mean()
        img /= img.std()
        X[0,] = img
        
        predict = model_seg.predict(X)
        
        image_id.append(i)
        mask.append(predict)
    for i in test.mask_path:
        mask_path.append(i)

    return pd.DataFrame({'image_path': image_id,'predicted_mask': mask,'mask_path':mask_path})

In [ ]:
data=prediction(Xtest,Res_Unet_model)

In [ ]:
data.head()

,image_path,predicted_mask,mask_path
0,/content/content/drive/MyDrive/CT_Data/CT_imag...,[[[[2.6382999e-15 3.6520727e-13 9.9376250e-14]...,/content/content/drive/MyDrive/CT_Data/Masks/2...
1,/content/content/drive/MyDrive/CT_Data/CT_imag...,[[[[2.5009241e-15 3.5917441e-13 9.4522656e-14]...,/content/content/drive/MyDrive/CT_Data/Masks/6...
2,/content/content/drive/MyDrive/CT_Data/CT_imag...,[[[[1.1936697e-15 3.7927406e-13 3.9285239e-14]...,/content/content/drive/MyDrive/CT_Data/Masks/3...
3,/content/content/drive/MyDrive/CT_Data/CT_imag...,[[[[5.0012874e-15 1.9625703e-12 2.9086986e-14]...,/content/content/drive/MyDrive/CT_Data/Masks/3...
4,/content/content/drive/MyDrive/CT_Data/CT_imag...,[[[[3.4140571e-15 3.2527217e-13 1.7729389e-13]...,/content/content/drive/MyDrive/CT_Data/Masks/4...


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
Res_Unet_model.fit(train_data, 
                  epochs = 120,
                  initial_epoch=41,
                  validation_data = val_data,
                  callbacks = [checkpointer, earlystopping, reduce_lr,history_logger]
                 )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


Epoch 42/120
391/391 [==============================] - ETA: 0s - loss: 0.1410 - tversky: 0.9240 - recall: 0.9489 - precision: 0.8836 - mean_io_u: 0.8645
Epoch 42: val_loss improved from inf to 0.20847, saving model to /content/drive/MyDrive/CT_res_unet_32/CT_res_unet_32.h5
391/391 [==============================] - 172s 397ms/step - loss: 0.1410 - tversky: 0.9240 - recall: 0.9489 - precision: 0.8836 - mean_io_u: 0.8645 - val_loss: 0.2085 - val_tversky: 0.8734 - val_recall: 0.8964 - val_precision: 0.8383 - val_mean_io_u: 0.8300 - lr: 0.0100
Epoch 43/120
391/391 [==============================] - ETA: 0s - loss: 0.1369 - tversky: 0.9273 - recall: 0.9504 - precision: 0.8855 - mean_io_u: 0.8652
Epoch 43: val_loss did not improve from 0.20847
391/391 [==============================] - 156s 398ms/step - loss: 0.1369 - tversky: 0.9273 - recall: 0.9504 - precision: 0.8855 - mean_io_u: 0.8652 - val_loss: 0.2143 - val_tversky: 0.8694 - val_recall: 0.8916 - val_precision: 0.8390 - val_mean_io_u: